In [125]:
from utils import first_block_of_day, last_block_of_day, rpc_to_parity
from utils import exeSQL, drop_table, exeMultipleSQL
from utils import map_args_group
from tqdm import tqdm_notebook as tqdm

date_start = '2017-01-01'
date_end = '2017-03-31'
block_start = first_block_of_day(int(date_start[:4]), int(date_start[5:7]), int(date_start[8:10]))
block_end = last_block_of_day(int(date_end[:4]), int(date_end[5:7]), int(date_end[8:10]))

In [106]:
table_name = "txs_20170101_20170331"
par = 20

In [143]:
print  (block_start, block_end)

2912407 3454528


In [107]:
def create_txs_table(table_name):
    create_txs_sql = """CREATE TABLE {} ( `id` int(11) NOT NULL AUTO_INCREMENT,
                `source` char(42) NOT NULL, `target` char(42) NOT NULL, `amount` varchar(32) NOT NULL, 
                `tx` char(66) NOT NULL, `block_num` int(11) NOT NULL, `tx_seq` int(11) NOT NULL, 
                PRIMARY KEY (`id`), KEY `block_num_index` (`block_num`), 
                FULLTEXT `target_index` (`target`), FULLTEXT `source_index` (`source`), 
                FULLTEXT `tx_index` (`tx`) ) ENGINE=InnoDB""".format(table_name)
    exeSQL(create_txs_sql, True)

In [140]:
recreate = True
if recreate:
    drop_table(table_name)
    create_txs_table(table_name)

In [109]:
def fetch_fields_from_txs(txs):
    entries = []
    for tx in txs:
        tmp_en = [tx['from'], tx['to'], tx['value'], tx['hash'], 
                  str(int(tx['blockNumber'], 16)), str(int(tx['transactionIndex'], 16))]
        entries.append(tmp_en)
    return entries

In [141]:
def insert_entries(entries, table_name):
    insert_entries_sqls = []
    for en in entries:
        sql = """INSERT INTO {} (source, target, amount, tx, block_num, tx_seq) 
            VALUES ('{}', '{}', '{}', '{}', '{}', '{}')""".format(table_name, *en)
        insert_entries_sqls.append(sql)
    exeMultipleSQL(insert_entries_sqls, True)

In [135]:
def fetch_reward_txs(block_num):
    def parse_action_reward(dict_obj, index):
        dict_action = dict_obj['action']
        if dict_action['rewardType'] == 'block': # reward-block txs are tagged by -1
            return dict_action['author'], dict_action['value'], -1
        elif dict_action['rewardType'] == 'uncle': # reward-uncle txs are tagged by -2, -3
            return dict_action['author'], dict_action['value'], index
        else:
            print ("Error................. neither block-reward nor uncle-reward {}".format(str(block_num)))
            return None
        
    method = 'trace_block'
    params = [hex(block_num)]
    actions = rpc_to_parity(method, params).json()['result']
    reward_entries = []
    uncle_index = -2
    for act in actions:
        if act['type'] == 'reward':
            t, v, i = parse_action_reward(act, uncle_index)
            tmp_index = ['0x0000000000000000000000000000000000000000', t, v, 'None', block_num, i]
            if i == -1:
                reward_entries.append(tmp_index)
            else:
                reward_entries.insert(0, tmp_index)
                uncle_index -= 1
    return reward_entries
            

In [136]:
def parse_blocks(block_start, block_end, table_name):
    for bn in tqdm(range(block_start, block_end)):
        method = 'eth_getBlockByNumber'
        params = [hex(bn), True]
        result = rpc_to_parity(method, params).json()['result']
        txs = result['transactions']
        entries = fetch_fields_from_txs(txs)
        reward_entries = fetch_reward_txs(bn)
        total_entries = reward_entries
        total_entries.extend(entries)
#         entries.insert(0, ['0x0000000000000000000000000000000000000000', result['miner'], str(3), 'None',
#                         str(int(result['number'], 16)), str(-1)])
        insert_entries(total_entries, table_name = table_name)

In [137]:
parse_blocks(4806832, 4806833, table_name=table_name)

INSERT INTO txs_20170101_20170331 (source, target, amount, tx, block_num, tx_seq) 
            VALUES ('0x0000000000000000000000000000000000000000', '0x9435d50503aee35c8757ae4933f7a0ab56597805', '0x29a2241af62c0000', 'None', '4806832', '-1')
INSERT INTO txs_20170101_20170331 (source, target, amount, tx, block_num, tx_seq) 
            VALUES ('0x0fd081e3bb178dc45c0cb23202069dda57064258', '0x6a2c747e4ee99962a6a81d18931ef96bbfc84132', '0x2985b825cff80000', '0x8b923db599b6e7cfcc5dbd3187a7d20f142d7bf58f1bd86a4035ddc50f6049a9', '4806832', '0')
INSERT INTO txs_20170101_20170331 (source, target, amount, tx, block_num, tx_seq) 
            VALUES ('0x0fd081e3bb178dc45c0cb23202069dda57064258', '0xcda36ed11428bf6637bc4de5550d967ac5a2b5db', '0xc52cf4b908c0000', '0x1d49e0e36b53c17cf87c8e6e91890b4132a39f923b0ed683e8b0926a15414ce0', '4806832', '1')
INSERT INTO txs_20170101_20170331 (source, target, amount, tx, block_num, tx_seq) 
            VALUES ('0xa30d8157911ef23c46c0eb71889efe6a648a41f7', '0x4

In [142]:
interval_start = block_start
flag = True

while(flag):
    if interval_start + 600 >= block_end + 1:
        interval_end = block_end + 1
        flag = False
    else:
        interval_end = interval_start + 600
    print ("Blocks from {} to {} are in process!".format(interval_start, interval_end))
    args_groups_list = []
    sub_range_len = (interval_end - interval_start) // par
    for i in range(par-1):
        tmp_group = [i*sub_range_len+interval_start, (i+1)*sub_range_len+interval_start, table_name]
        args_groups_list.append(tmp_group)
    tmp_group = [(par-1)*sub_range_len+interval_start, interval_end, table_name]
    args_groups_list.append(tmp_group)
    print (map_args_group(args_groups_list, parse_blocks))
    interval_start += 600

Blocks from 2912407 to 2913007 are in process!
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Blocks from 2913007 to 2913607 are in process!
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Blocks from 2913607 to 2914207 are in process!
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Blocks from 2914207 to 2914807 are in process!
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Blocks from 2914807 to 2915407 are in process!


Process ForkPoolWorker-212:
Process ForkPoolWorker-207:
Process ForkPoolWorker-219:
Process ForkPoolWorker-208:
Process ForkPoolWorker-213:


KeyboardInterrupt: 

Process ForkPoolWorker-206:
Process ForkPoolWorker-222:
Process ForkPoolWorker-223:
Process ForkPoolWorker-215:
Process ForkPoolWorker-220:
Process ForkPoolWorker-209:
Process ForkPoolWorker-217:
Process ForkPoolWorker-210:
Process ForkPoolWorker-205:
Process ForkPoolWorker-221:
Process ForkPoolWorker-211:
Process ForkPoolWorker-218:
Process ForkPoolWorker-224:
Process ForkPoolWorker-214:
Process ForkPoolWorker-216:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessi

  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):

During handling of the above exception, another exception occurred:

  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "<ipython-input-141-fb5f330047b2>", line 7, in insert_entries
    exeMultipleSQL(insert_entries_sqls, True)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "<ipython-input-141-fb5f330047b2>", line 7, in insert_entries
    exeMultipleSQL(insert_entries_sqls, True)
  File "<ipython-input-141-fb5f330047b2>", line 7, in insert_entries
    exeMultipleSQL(insert_entries_sqls, True)

During handling of the above exception, another exception occurred:


During handling of the above exception, another exception occurred:


During handling of the above 

  File "/usr/lib/python3.5/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "<ipython-input-136-f69722aa06b9>", line 5, in parse_blocks
    result = rpc_to_parity(method, params).json()['result']
  File "/home/seafooler/.local/lib/python3.5/site-packages/pymysql/connections.py", line 656, in _read_packet
    packet_header = self._read_bytes(4)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/seafooler/.local/lib/python3.5/site-packages/pymysql/connections.py", line 656, in _read_packet
    packet_header = self._read_bytes(4)
  File "<ipython-input-136-f69722aa06b9>", line 5, in parse_blocks
    result = rpc_to_parity(method, params).json()['result']
  File "<ipython-input-136-f69722aa06b9>", line 5, in parse_blocks
    result = rpc_to_parity(method, params).json()['result']
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
  

KeyboardInterrupt
  File "/usr/lib/python3/dist-packages/requests/sessions.py", line 468, in request
    resp = self.send(prep, **send_kwargs)
  File "/usr/lib/python3/dist-packages/requests/sessions.py", line 468, in request
    resp = self.send(prep, **send_kwargs)
  File "/home/seafooler/parse_data_from_parity/utils.py", line 55, in rpc_to_parity
    response = session.post(rpc_ip_port, json=payload, headers=headers)
KeyboardInterrupt
  File "/home/seafooler/parse_data_from_parity/utils.py", line 55, in rpc_to_parity
    response = session.post(rpc_ip_port, json=payload, headers=headers)
  File "<ipython-input-135-265e228304bf>", line 14, in fetch_reward_txs
    actions = rpc_to_parity(method, params).json()['result']
  File "<ipython-input-135-265e228304bf>", line 14, in fetch_reward_txs
    actions = rpc_to_parity(method, params).json()['result']
  File "/home/seafooler/parse_data_from_parity/utils.py", line 55, in rpc_to_parity
    response = session.post(rpc_ip_port, json=payloa

  File "/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3/dist-packages/requests/adapters.py", line 376, in send
    timeout=timeout
  File "/usr/lib/python3/dist-packages/requests/adapters.py", line 376, in send
    timeout=timeout
  File "/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3/dist-packages/requests/adapters.py", line 376, in send
    timeout=timeout
  File "/usr/lib/python3/dist-packages/requests/sessions.py", line 576, in send
    r = adapter.send(request, **kwargs)
KeyboardInterrupt
  File "/usr/lib/python3/dist-packages/requests/adapters.py", line 376, in send
    timeout=timeout
  File "/usr/lib/python3/dist-packages/requests/sessions.py", line 576, in send
    r = adapter.send(request, **kwargs)
  File "/usr/lib/python3/dist-packages/requests/sessio

  File "/usr/lib/python3.5/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.5/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
KeyboardInterrupt
  File "/usr/lib/python3.5/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.5/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.5/http/client.py", line 277, in _read_status
    if not version.startswith("HTTP/"):
  File "/usr/lib/python3.5/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.5/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.5/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/